<a href="https://colab.research.google.com/github/Ismail-Armutcu/Algorithms-for-Interactive-Sytems/blob/main/MMI513_PA4_Tournaments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Programming Assignment #4**

## Fall 2023/24 Semester

1. Your professor, while looking back at how badly he implemented dictionary structure that holds the player data feels frustrated. He thinks it may just be much easier to implement a `Player` class with appropriate variables (`name`, `rank`, `peerWinner`), the getter and setter functions (`getrank`, `setrank` etc.) and perhaps other utilities like `copy` Your first task is to implement this class.

2. You will then implement `unrankedplayers()`, `initialrankadjustment()` and `pyramidmatch()` functions.

3. Finally, you will implement `kingofthehill()` tournament given on p. 112 (Algorithm 5.5) of Jouni and Smed.

**GRADING:**
- `Player` class is implemented (5/100)
- `unrankedplayers()` using `Player` class (5/100)
- `initialrankadjustment()` using `Player` class (5/100)
- `pyramidmatch()` using the `Player` class (5/100)
- `kingofthehill()` function works with default arguments (40/100)
- Code carries out necessary checks on inputs via proper assertions (20/100)
- Code is properly commented, explained and documented (20/100)


**Notes:**

- You are free to use chatGPT for help. However, you should not use code generated by chatGPT directly and you should mention which part of the code is chatGPT in the comments. Your code will be checked for verbatim copying from chatGPT.
- You are not allowed to use external modules/libraries in your implementations except those already in the notebook, but you can use them for testing your code.
- You will upload your submission as a Python notebook to ODTUClass.

In [1]:
import numpy as np

In [2]:
class Player:

  def __init__(self, name, rank = 0,peerStatus = False):
    assert(type(peerStatus) == bool)
    assert(rank == None or rank >=0)
    self.name = name
    self.rank = rank
    self.peerWinner = peerStatus

  def __str__(self):
    return f"Name:{self.name} Rank:{self.rank} PeerWinner:{self.peerWinner}"
  def __repr__(self):
    return str(self)
  def __eq__(self, object2):
    if(object2 == None):
      return False
    elif(self.name == object2):
      return True
    elif(self.name == object2.name):
      return True
    else:
      return False
  def __ne__(self, object2):
    return self.name != object2.name

  def setRank(self,rank):
    assert( rank >=0 or rank == None)
    self.rank = rank

  def getRank(self):
    return self.rank

  def setPeerWinner(self, status):
    assert(type(status) == bool)
    self.peerWinner = status

  def getPeerWinner(self):
    return self.peerWinner

class PlayerList:
  def __init__(self):
    self.players = []


  def __str__(self):
    Name = []
    Rank = []
    PeerWinner = [9]
    for p in self.players:
      Name.append(p.name)
      Rank.append(p.getRank())
      PeerWinner.append(p.getPeerWinner())
    return f"Name:{Name} Rank:{Rank} PeerWinner:{PeerWinner}"
  def __repr__(self):
    return str(self)
  def __getitem__(self,item):
    return self.players[item]
  def index(self,player,strName):





In [3]:
def unrankedplayers(n):
    P = []
    for ind in range(n):
        p = Player(name = 'P' + str(ind), rank = None)
        P.append(p)

    return P

def enumeration(P):
    lst = np.arange(len(P))
    order = np.random.permutation(lst)
    M = []

    for ind in order:
      M.append(P[ind])

    return M

def rankeds(P, r):
    W = []
    for p in P:
      if p.getRank() == r:
        W.append(p)
    return W

def setrank(P, index, r):
    R = P.copy()
    R[index].setRank(r)
    return R

def match(p, q, thr, tieflag=True):
    toss = np.random.rand(1)
    upC = 1
    downC = 1
    if tieflag: # If Tie is allowed
      upC = 1.1
      downC = 0.9
    if thr * downC < toss < thr * 1.1: # If the random number is around thr, result is a tie
        result = None # Tie
    elif toss > thr * upC:
        result = p
    else:
        result = q
    return result

def ladder_match(P, p, q):
    m = match(p, q, 0.5)
    if m == None or m == p:
        return P
    else:
        indp = P.index(p)
        indq = P.index(q)
        rankp = P[indp].getRank()
        rankq = P[indq].getRank()
        if rankq == None:
            rankq = rankp
        P[indp].setRank(rankq)
        P[indq].setRank(rankp)
        return P

def initial_rank_adjustment(P, S):
    assert len(P) == sum(S)
    m = len(S)
    R = P.copy()
    R = enumeration(R)
    for ind in range(S[0]):
      R[ind].setRank(0) # The initial champion(s)
    c =  S[0]

    for rank in range(1, m):
        W = rankeds(R, rank - 1) # The runners-up
        Mp = enumeration(W)
        for ind in range(S[rank]):
            R[c+ind].setRank(rank)
            jnd = ind % len(Mp)

            if Mp[jnd].getRank() != rank:
              R = ladder_match(R, Mp[jnd], M[c + ind])
        c += S[rank]
    return R

In [4]:
P = unrankedplayers(11)
x = Player("P0",None,False)

print(P.index(x))
S = [1, 2, 3, 5] # Ranking structure: One champion, 2 second places, etc.
M = enumeration(P)
print(M)
R = initial_rank_adjustment(P, S)
print(R)



[Name:P0 Rank:None PeerWinner:False, Name:P1 Rank:None PeerWinner:False, Name:P2 Rank:None PeerWinner:False, Name:P3 Rank:None PeerWinner:False, Name:P4 Rank:None PeerWinner:False, Name:P5 Rank:None PeerWinner:False, Name:P6 Rank:None PeerWinner:False, Name:P7 Rank:None PeerWinner:False, Name:P8 Rank:None PeerWinner:False, Name:P9 Rank:None PeerWinner:False, Name:P10 Rank:None PeerWinner:False]
0
[Name:P5 Rank:None PeerWinner:False, Name:P7 Rank:None PeerWinner:False, Name:P0 Rank:None PeerWinner:False, Name:P8 Rank:None PeerWinner:False, Name:P3 Rank:None PeerWinner:False, Name:P1 Rank:None PeerWinner:False, Name:P2 Rank:None PeerWinner:False, Name:P4 Rank:None PeerWinner:False, Name:P9 Rank:None PeerWinner:False, Name:P10 Rank:None PeerWinner:False, Name:P6 Rank:None PeerWinner:False]
[Name:P10 Rank:2 PeerWinner:False, Name:P9 Rank:1 PeerWinner:False, Name:P7 Rank:1 PeerWinner:False, Name:P0 Rank:2 PeerWinner:False, Name:P5 Rank:0 PeerWinner:False, Name:P4 Rank:2 PeerWinner:False, Na

In [14]:
def pyramid_match(P, p, q):
  playerPindex = Player('P'+str(p))
  playerQindex = Player('P'+str(p))
  pi = P.index(playerPindex)
  qi = P.index(playerQindex)
  rank_p = P[pi].getRank()
  peerWinner_p = P[pi].getPeerWinner()
  rank_q = P[qi].getRank()
  peerWinner_q = P[qi].getPeerWinner()

  print((rank_p == rank_q) and (not peerWinner_q))
  print((rank_p == rank_q - 1) and peerWinner_q)

  print((rank_p == rank_q) and (not peerWinner_q))
  print((rank_p == rank_q - 1) and peerWinner_q)

  if not ((rank_p == rank_q) and (not peerWinner_q)) | ((rank_p == rank_q - 1) and peerWinner_q):
    print('No match was played! Ranks do not match the conditions for pyramid match!')
    return P

  R = P.copy() # We copy the player dictionary first
  x = R[pi]
  print(x)
  print(R)
  R.pop(x)
  print(R)
  R['P'].pop(qi)
  R['rnks'].pop(pi)
  R['rnks'].pop(qi)
  R['peerWinner'].pop(pi)
  R['peerWinner'].pop(qi)

  print(R)

  # We do not have the two players in our set of players now.
  m = match(p, q, 0.5) # Let them compete

  if (rank_p == rank_q): # Peer match
    if (((m == p) and (not peerWinner_p)) or (((m == q) or m == None) and peerWinner_p)):
      pp = p
      rank_pp = rank_p
      peerWinner_pp = (m == p)
    else:
      pp = p
      rank_pp = rank_p
      peerWinner_pp = peerWinner_p
    if (m == q):
      qp = q
      rank_qp = rank_q
      peerWinner_qp = True
    else:
      qp = q
      rank_qp = rank_q
      peerWinner_qp = peerWinner_q

    R['P'].append('P'+str(pp))
    R['P'].append('P'+str(qp))
    R['rnks'].append(rank_pp)
    R['rnks'].append(rank_qp)
    R['peerWinner'].append(peerWinner_pp)
    R['peerWinner'].append(peerWinner_qp)
    return R
  else: # Rank challenge match
    qp = q
    rank_qp = rank_q
    peerWinner_qp = False
    if (m == p) or m == None:
      R['P'].append('P'+str(p))
      R['P'].append('P'+str(qp))
      R['rnks'].append(rank_p)
      R['rnks'].append(rank_qp)
      R['peerWinner'].append(peerWinner_p)
      R['peerWinner'].append(peerWinner_qp)
      return R
    else:
      pp = p
      peerWinner_pp = False
      rank_pp = rank_q
      rank_qp = rank_p
      R['P'].append('P'+str(pp))
      R['P'].append('P'+str(qp))
      R['rnks'].append(rank_pp)
      R['rnks'].append(rank_qp)
      R['peerWinner'].append(peerWinner_pp)
      R['peerWinner'].append(peerWinner_qp)
      return R

In [15]:
n = 11
P = unrankedplayers(n)
S = [1, 2, 3, 5]
R = initial_rank_adjustment(P, S)
# Appending the peerWinner field to R makes it into pyramid structure
for r in R:
  r.setPeerWinner(False)
print(R)

[Name:P4 Rank:0 PeerWinner:False, Name:P9 Rank:2 PeerWinner:False, Name:P5 Rank:1 PeerWinner:False, Name:P1 Rank:1 PeerWinner:False, Name:P7 Rank:2 PeerWinner:False, Name:P10 Rank:2 PeerWinner:False, Name:P2 Rank:3 PeerWinner:False, Name:P8 Rank:3 PeerWinner:False, Name:P3 Rank:3 PeerWinner:False, Name:P6 Rank:3 PeerWinner:False, Name:P0 Rank:3 PeerWinner:False]


In [16]:
R = pyramid_match(R, 0, 5)
print(R)

True
False
True
False
Name:P0 Rank:3 PeerWinner:False
[Name:P4 Rank:0 PeerWinner:False, Name:P9 Rank:2 PeerWinner:False, Name:P5 Rank:1 PeerWinner:False, Name:P1 Rank:1 PeerWinner:False, Name:P7 Rank:2 PeerWinner:False, Name:P10 Rank:2 PeerWinner:False, Name:P2 Rank:3 PeerWinner:False, Name:P8 Rank:3 PeerWinner:False, Name:P3 Rank:3 PeerWinner:False, Name:P6 Rank:3 PeerWinner:False, Name:P0 Rank:3 PeerWinner:False]


TypeError: 'Player' object cannot be interpreted as an integer